In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import os 
# Impor the OpenWeatherMap API key
from nerl_apikey import nerl_apikey


In [34]:
nerl_apikey = nerl_apikey

In [35]:
cities_data = [
    {"NAME": "New York", "coordinates": (40.712776, -74.005974)},
    {"NAME": "Los Angeles", "coordinates": (34.052235, -118.243683)}]

In [36]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(cities_data)
df

,NAME,coordinates
0,New York,"(40.712776, -74.005974)"
1,Los Angeles,"(34.052235, -118.243683)"


In [37]:

# Expand the 'coordinates' column into 'latitude' and 'longitude'
df[['Latitude', 'Longitude']] = pd.DataFrame(df['coordinates'].tolist(), index=df.index)
df

,NAME,coordinates,Latitude,Longitude
0,New York,"(40.712776, -74.005974)",40.712776,-74.005974
1,Los Angeles,"(34.052235, -118.243683)",34.052235,-118.243683


In [38]:
df.drop(df.columns[[2, 3]], axis=1, inplace=True)
df

,NAME,coordinates
0,New York,"(40.712776, -74.005974)"
1,Los Angeles,"(34.052235, -118.243683)"


from urllib.parse import quote

route_param = "LINESTRING(-74.0060152 40.7127281,-75.2437 39.8730)"
encoded_route_param = quote(route_param)

print(encoded_route_param)

In [40]:
import requests
from urllib.parse import quote

def fetch_ev_charging_stations(api_key, route_param):
    base_url = "https://developer.nrel.gov/api/alt-fuel-stations/v1/nearby-route.json"
    params = {
        "api_key": api_key,
        "route": route_param,  # Use the encoded LINESTRING directly
        "fuel_type": "ELEC",
        "limit": "all",
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}, Response: {response.text}")
        return None

# Example usage
api_key = 'YOUR API KEY HERE'  # Replace with your actual NREL API key
route_param = "LINESTRING(-74.0060152 40.7127281,-75.2437 39.8730)" #NY to PHL, can chage to LA or wherever for more data
#encoded_route_param = quote(route_param)

# Pass the encoded LINESTRING to the function
data = fetch_ev_charging_stations(api_key, route_param)

if data:
    stations = data.get('fuel_stations', [])
    for station in stations:
        print(f"Station Name: {station.get('station_name')}")
        print(f"Address: {station.get('street_address')}")
        print(f"City: {station.get('city')}")
        print(f"State: {station.get('state')}")
        print(f"ZIP: {station.get('zip')}")
        print(f"Lattitude: {station.get('latitude')}")
        print(f"Longitude: {station.get('longitude')}")
        print("----------")
else:
    print("No data returned")

Failed to fetch data: 403, Response: {
  "error": {
    "code": "API_KEY_INVALID",
    "message": "An invalid api_key was supplied. Get one at https://developer.nrel.gov/signup/"
  }
}
No data returned


In [16]:
stations_list = data.get('fuel_stations', [])

# Convert the list of dictionaries (stations) into a pandas DataFrame
stations_df = pd.DataFrame(stations_list)

# Display the first few rows of the DataFrame to verify it looks correct
print(stations_df.head())

  access_code                           access_days_time access_detail_code  \
0     private  Contact station for hours of availability               None   
1      public                                       None               None   
2      public                                       None               None   
3      public                                       None               None   
4      public                             24 hours daily               None   

  cards_accepted date_last_confirmed expected_date fuel_type_code  \
0           None          2023-06-12          None           ELEC   
1           None          2022-10-06          None           ELEC   
2           None          2022-10-06          None           ELEC   
3           None          2022-10-06          None           ELEC   
4           None          2021-09-10          None           ELEC   

  groups_with_access_code      id maximum_vehicle_class  ... nps_unit_name  \
0                 Private  26221

In [17]:
pd.DataFrame.to_csv(stations_df,"./NERL_extractt.csv", index=True)

In [18]:
file_path = "./NERL_extractt.csv"

# Read the CSV file into a DataFrame
df_all = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df_all.head())

   Unnamed: 0 access_code                           access_days_time  \
0           0     private  Contact station for hours of availability   
1           1      public                                        NaN   
2           2      public                                        NaN   
3           3      public                                        NaN   
4           4      public                             24 hours daily   

  access_detail_code cards_accepted date_last_confirmed expected_date  \
0                NaN            NaN          2023-06-12           NaN   
1                NaN            NaN          2022-10-06           NaN   
2                NaN            NaN          2022-10-06           NaN   
3                NaN            NaN          2022-10-06           NaN   
4                NaN            NaN          2021-09-10           NaN   

  fuel_type_code groups_with_access_code      id  ... nps_unit_name  \
0           ELEC                 Private  262217  ...    

In [23]:
df_all.columns

Index(['Unnamed: 0', 'access_code', 'access_days_time', 'access_detail_code',
       'cards_accepted', 'date_last_confirmed', 'expected_date',
       'fuel_type_code', 'groups_with_access_code', 'id',
       'maximum_vehicle_class', 'open_date', 'owner_type_code',
       'restricted_access', 'status_code', 'facility_type', 'station_name',
       'station_phone', 'updated_at', 'geocode_status', 'latitude',
       'longitude', 'city', 'country', 'intersection_directions', 'plus4',
       'state', 'street_address', 'zip', 'bd_blends', 'cng_dispenser_num',
       'cng_fill_type_code', 'cng_has_rng', 'cng_psi', 'cng_renewable_source',
       'cng_total_compression', 'cng_total_storage', 'cng_vehicle_class',
       'e85_blender_pump', 'e85_other_ethanol_blends', 'ev_connector_types',
       'ev_dc_fast_num', 'ev_level1_evse_num', 'ev_level2_evse_num',
       'ev_network', 'ev_network_web', 'ev_other_evse', 'ev_pricing',
       'ev_renewable_source', 'ev_workplace_charging', 'hy_is_retail',
 

In [24]:
df_filter = df_all[['station_name','street_address', 'city','state', 'zip', 'ev_network','latitude','longitude']]
df_filter

,station_name,street_address,city,state,zip,ev_network,latitude,longitude
0,Duggal Visual Solutions,63 Flushing Ave,Brooklyn,NY,11205,AMPUP,40.698625,-73.978621
1,Little Man Parking (Queens West) - Tesla Desti...,4-76 47th Ave,Long Island City,NY,11109,Tesla Destination,40.745627,-73.955638
2,iPark 59-63 Allen Street - Tesla Destination,59-63 Allen St,New York,NY,10002,Tesla Destination,40.717143,-73.991597
3,iPark 34 N 7th Street 2 - Tesla Destination,34 N 7th St,Brooklyn,NY,11249,Tesla Destination,40.720708,-73.962310
4,Revel Brooklyn Superhub,467 Marcy Ave,Brooklyn,NY,11206,EVGATEWAY,40.699334,-73.949665
...,...,...,...,...,...,...,...,...
1055,PPA AIRPORT EV GARAGE C STAT2,1 Arrivals Rd,Philadelphia,PA,19153,ChargePoint Network,39.878084,-75.241619
1056,PPA AIRPORT EV GARAGE C STAT1,1 Arrivals Rd,Philadelphia,PA,19153,ChargePoint Network,39.878061,-75.241638
1057,Wawa Folcroft,1550 Chester Pike,Folcroft,PA,19032,eVgo Network,39.903892,-75.283386
1058,Robin Ford,100 N. MacDade Blvd,Glenolden,PA,19036,Blink Network,39.907034,-75.289394


In [25]:
#import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [31]:
host = ''
database = 'evchargerinst'
user = ''
password = ''
port = '5432'

In [27]:
df_filter.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_22096\829654206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)


In [28]:
df_filter

,station_name,street_address,city,state,zip,ev_network,lat,lon
0,Duggal Visual Solutions,63 Flushing Ave,Brooklyn,NY,11205,AMPUP,40.698625,-73.978621
1,Little Man Parking (Queens West) - Tesla Desti...,4-76 47th Ave,Long Island City,NY,11109,Tesla Destination,40.745627,-73.955638
2,iPark 59-63 Allen Street - Tesla Destination,59-63 Allen St,New York,NY,10002,Tesla Destination,40.717143,-73.991597
3,iPark 34 N 7th Street 2 - Tesla Destination,34 N 7th St,Brooklyn,NY,11249,Tesla Destination,40.720708,-73.962310
4,Revel Brooklyn Superhub,467 Marcy Ave,Brooklyn,NY,11206,EVGATEWAY,40.699334,-73.949665
...,...,...,...,...,...,...,...,...
1055,PPA AIRPORT EV GARAGE C STAT2,1 Arrivals Rd,Philadelphia,PA,19153,ChargePoint Network,39.878084,-75.241619
1056,PPA AIRPORT EV GARAGE C STAT1,1 Arrivals Rd,Philadelphia,PA,19153,ChargePoint Network,39.878061,-75.241638
1057,Wawa Folcroft,1550 Chester Pike,Folcroft,PA,19032,eVgo Network,39.903892,-75.283386
1058,Robin Ford,100 N. MacDade Blvd,Glenolden,PA,19036,Blink Network,39.907034,-75.289394


In [29]:
df_filter.dtypes

station_name       object
street_address     object
city               object
state              object
zip                 int64
ev_network         object
lat               float64
lon               float64
dtype: object

In [1]:
# Construct the connection string
db_connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
# Define the schema name and view name
schema_name = 'evChargers'
table_name = 'TO BE ALL RECORDS TABLE'
# Write the DataFrame as a view to the database
#df_filter.to_sql(table_name, con=create_engine(db_connection_str), schema=schema_name, if_exists='replace', index=False)

NameError: name 'user' is not defined